In [3]:
import trl
print("trl version:", trl.__version__)

trl version: 0.19.1


In [4]:
from dataclasses import dataclass, field, asdict

In [9]:
import logging
from datasets import load_from_disk

training_dataset = load_from_disk("cache")
print(f"training_dataset: {training_dataset}")

training_dataset: DatasetDict({
    train: Dataset({
        features: ['image', 'messages'],
        num_rows: 281334
    })
})


In [10]:
training_dataset = training_dataset["train"]
print(training_dataset[0].keys())

dict_keys(['image', 'messages'])


In [11]:
for i in range(min(3, len(training_dataset))):
        img = training_dataset[i]['image']
        print(f"Sample {i}: {type(img)}")

Sample 0: <class 'NoneType'>
Sample 1: <class 'NoneType'>
Sample 2: <class 'NoneType'>


In [1]:
images = ["obama.png"]
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Who is this?"},
            {"type": "image"}
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "Barack Obama"}
        ]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What is he famous for?"}
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "He is the 44th President of the United States."}
        ]
    }
]

from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
print(processor.apply_chat_template(messages, tokenize=False))

/home/ubuntu/ntu/brandon/prm-training-code/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


USER: <image>
Who is this? ASSISTANT: Barack Obama USER: What is he famous for? ASSISTANT: He is the 44th President of the United States. 


In [2]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")
print(processor.apply_chat_template(messages, tokenize=False))

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Who is this?<|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant
Barack Obama<|im_end|>
<|im_start|>user
What is he famous for?<|im_end|>
<|im_start|>assistant
He is the 44th President of the United States.<|im_end|>

